In [227]:
import json
from collections import defaultdict

In [228]:

with open('./HumanEval_Py_original.jsonl', 'r') as f:
    data = f.readlines()
    original_data = [json.loads(line) for line in data]

mapping = defaultdict(list)
for d in original_data:
    id = d['task_id'].split('/')[1]
    mapping[id].append(d)


In [229]:
from execution import check_correctness

In [230]:
file_name = 'StarCoder_Output_Py_0.2'
with open(f'./Output/{file_name}.json', 'r') as f:
    data = json.load(f)


In [231]:
split_tokens = ["\n'''","\n\"\"\"'"]

In [232]:
def clear_generated_code(data):
    data = data.split('<|endoftext|>')[0]   

    for token in split_tokens:
        data = data.split(token)[0]
        
    lines = data.split('\n')
    new_data = ''
    for line in lines:
        if 'def ' in line:
            continue
        if not line.startswith(' '):
            line = '    '+line
        new_data += line+'\n'

    

    return new_data


In [233]:
new_data = []
for d in data:
    id = d['id']

    original_id = id.split('_')[0]
    original_item = mapping[original_id][0]
    problem ={
        'task_id': id,
        'test': original_item['test'],
        'entry_point': original_item['entry_point'],
        'prompt': d['prompt']+'\n',
    }
    # for i, c in enumerate(d['output']['choices']):
    for i, c in enumerate(d['output']):
        # print(i)
        # completion = clear_generated_code(c['message']['content'])
        # completion = c['message']['content']
        completion = c['text'].split('<|endoftext|>')[0]
        problem['completion_'+str(i)] = completion
        with open(f'./Files/{id}_{str(i)}.py', 'w') as f:
            f.write(completion)

#         result = check_correctness(problem, completion, 3.0)
#         problem['result_'+str(i)] = result
#     new_data.append(problem)
#         # print(problem)
# with open(f'./Output/{file_name}_result.jsonl', 'w') as f:
#     for d in new_data:
#         f.write(json.dumps(d)+'\n')